# Part 1

Integer Optimization Problem

cost = cost_a * n_a + cost_b * n_b
constaints
x_price = delta_xa * n_a + delta_xb * nb
y_price = delta_ya * n_a + delta_xb * nb

In [ ]:
from scipy.optimize import linprog
import numpy as np

COSTS = {"a": 3, "b": 1}
from sympy import symbols
from sympy.matrices import Matrix


def find_optimun(prize_loc, costs, delta_x, delta_y, verbose=False):
    cost_a = costs["a"]
    cost_b = costs["b"]

    delta_xa = delta_x["a"]
    delta_xb = delta_x["b"]

    delta_ya = delta_y["a"]
    delta_yb = delta_y["b"]

    # c = [cost_a, cost_b]
    # A_eq = [[delta_xa, delta_xb], [delta_ya, delta_yb]]
    # b_eq = [prize_loc["x"], prize_loc["y"]]
    matrix_num = [[delta_xa, delta_xb], [delta_ya, delta_yb]]
    # first, try to solve algebraically
    if np.linalg.det(matrix_num) != 0:
        # need to be solved with sympy to avoid floating point errors
        A = Matrix(matrix_num)
        b = Matrix([prize_loc["x"], prize_loc["y"]])

        movements = [float(x) for x in A.solve(b)]

        # movements = [round(x, 6) for x in np.linalg.solve(A_eq, b_eq)]
        na, nb = movements
        if verbose:
            print(movements)
        if na >= 0 and na.is_integer() and nb >= 0 and nb.is_integer():
            is_optimal = True
            movements = [int(x) for x in movements]
            tokens = cost_a * movements[0] + cost_b * movements[1]
            return is_optimal, movements, tokens
        else:
            return False, None, None
    else:
        raise NotImplementedError("Infinite solutions")
    # scipy not working due to floating point errors
    # else:
    #     print("Infinite sols")
    #     max_na = min(prize_loc["x"] // delta_xa, prize_loc["y"] // delta_ya)
    #     max_nb = min(prize_loc["x"] // delta_xb, prize_loc["y"] // delta_yb)
    #     res = linprog(
    #         c,
    #         A_eq=A_eq,
    #         b_eq=b_eq,
    #         bounds=[(0, max_na), (0, max_nb)],
    #         integrality=[1, 1],
    #     )

    #     # optimal?
    #     is_optimal = res.success
    #     if is_optimal:
    #         tokens = int(res.fun)
    #         movements = [int(x) for x in res.x]
    #         return is_optimal, movements, tokens
    #     else:
    #         return is_optimal, None, None


def parse_input(input: str):
    lines = input.split("\n")
    problems = []
    for i in range(0, len(lines), 4):
        # print(lines[i])
        delta_x = {}
        delta_x["a"] = int(lines[i].split("X+")[-1].split(",")[0])
        delta_x["b"] = int(lines[i + 1].split("X+")[-1].split(",")[0])
        delta_y = {}
        delta_y["a"] = int(lines[i].split("Y+")[-1])
        delta_y["b"] = int(lines[i + 1].split("Y+")[-1])
        # extract from "Prize: X=8400, Y=5400"
        l_prices = lines[i + 2]
        x_loc = int(l_prices.split("X=")[-1].split(",")[0])
        y_loc = int(l_prices.split("Y=")[-1])
        prize_loc = {"x": x_loc, "y": y_loc}
        # print(lines[i + 3])
        problems.append((prize_loc, delta_x, delta_y))

    return problems


def get_total_tokens(input_problem: str, verbose=False):
    total_tokens = 0
    for prize_loc, delta_x, delta_y in parse_input(input_problem):
        is_optimal, movements, tokens = find_optimun(prize_loc, COSTS, delta_x, delta_y)
        # return movements
        if is_optimal:
            total_tokens += tokens
            if verbose:
                print(f"{movements[0]} {movements[1]} {tokens}")
        else:
            if verbose:
                print("No solution")

    return total_tokens


In [2]:
input_small = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

assert get_total_tokens(input_small, verbose=True) == 480

80 40 280
No solution
38 86 200
No solution


In [3]:
with open("input.txt") as f:
    input_large = f.read()

assert get_total_tokens(input_large) == 37297

In [4]:
def get_total_tokens_2(input_problem: str, verbose=False):
    total_tokens = 0
    for prize_loc, delta_x, delta_y in parse_input(input_problem):
        # add 10000000000000 to prize coordinates
        prize_loc["x"] += 10000000000000
        prize_loc["y"] += 10000000000000
        is_optimal, movements, tokens = find_optimun(prize_loc, COSTS, delta_x, delta_y)
        if is_optimal:
            total_tokens += tokens
            if verbose:
                print(f"{movements[0]} {movements[1]} {tokens}")
        else:
            if verbose:
                print("No solution")

    return total_tokens


assert get_total_tokens_2(input_large, verbose=False) == 83197086729371